# 动静转换

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/zh_cn/model_train/program_form/mindspore_jit.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/zh_cn/model_train/program_form/mindspore_jit.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.svg)](https://gitee.com/mindspore/docs/blob/master/docs/mindspore/source_zh_cn/model_train/program_form/jit.ipynb)

## 概述

当前在业界支持动态图和静态图两种模式，动态图通过解释执行，具有动态语法亲和性，表达灵活；静态图使用jit编译优化执行，偏静态语法，在语法上有较多限制。动态图和静态图的编译流程不一致，语法约束不一致。MindSpore针对动态图和静态图模式，首先统一API表达，在两种模式下使用相同的API；其次统一动态图和静态图的底层微分机制。

![dynamic](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/docs/mindspore/source_zh_cn/model_train/program_form/images/dynamic.png)

## 动态图和静态图互相转换

在MindSpore中，我们可以通过控制模式输入参数来切换执行使用动态图还是静态图。例如：

In [5]:
import mindspore as ms
ms.set_context(mode=ms.PYNATIVE_MODE)

由于在静态图下，对于Python语法有所限制，因此从动态图切换成静态图时，需要符合静态图的语法限制，才能正确使用静态图来进行执行。更多静态图的语法限制可以参考[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)。

## 动静结合

MindSpore支持在动态图下使用静态编译的方式来进行混合执行，通过使用jit修饰需要用静态图来执行的函数对象，即可实现动态图和静态图的混合执行，更多jit的使用可参考[jit文档](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/accelerate_with_static_graph.html#基于装饰器的开启方式)。

例如：

In [6]:
import numpy as np
import mindspore as ms
import mindspore.nn as nn

class AddMulMul(nn.Cell):
    def __init__(self):
        super(AddMulMul, self).__init__()
        self.param = ms.Parameter(ms.Tensor(0.5, ms.float32))

    @ms.jit
    def construct(self, x):
        x = x + x
        x = x * self.param
        x = x * x
        return x

class CellCallSingleCell(nn.Cell):
    def __init__(self):
        super(CellCallSingleCell, self).__init__()
        self.conv = nn.Conv2d(1, 2, kernel_size=2, stride=1, padding=0, weight_init="ones", pad_mode="valid")
        self.bn = nn.BatchNorm2d(2, momentum=0.99, eps=0.00001, gamma_init="ones")
        self.relu = nn.ReLU()
        self.add_mul_mul = AddMulMul()

    def construct(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.add_mul_mul(x)
        x = self.relu(x)
        return x

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
inputs = ms.Tensor(np.ones([1, 1, 2, 2]).astype(np.float32))
net = CellCallSingleCell()
out = net(inputs)
print(out)

[[[[15.99984]]

  [[15.99984]]]]


## 静态图语法增强技术

在MindSpore静态图模式下，用户编写程序时需要遵循MindSpore[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)，语法使用存在约束限制。而在动态图模式下，Python脚本代码会根据Python语法进行执行，用户可以使用任意Python语法。可以看出，静态图和动态图的语法约束限制是不同的。

JIT Fallback是从静态图的角度出发考虑静态图和动态图的统一。在编译过程中发现不支持的语法时，将该语法Fallback到Python解释器进行解释执行。通过JIT Fallback特性，静态图可以支持尽量多的动态图语法，使得静态图提供接近动态图的语法使用体验，从而实现动静统一。

在图模式场景下，MindSpore框架在图编译过程中遇到不支持语法或不支持符号时会报错，多数是在类型推导阶段。图编译阶段首先对用户编写的Python源码进行解析，然后执行后续的静态分析、类型推导、优化等步骤。因此，JIT Fallback特性需要预先检测不支持语法。常见的不支持语法主要包括：调用第三方库的方法、调用类名来创建对象、调用未支持的Python内置函数等。对不支持的语法Fallback到Python解释器进行解释执行。由于图模式采用[中间表示MindIR](https://www.mindspore.cn/docs/zh-CN/master/design/all_scenarios.html#中间表示mindir)，需要将解释执行的语句转换到中间表示，记录下解释器需要的信息。

下面主要介绍使用JIT Fallback扩展支持的静态图语法。JIT语法支持级别选项jit_syntax_level的默认值为`LAX`，即使用JIT Fallback的能力扩展静态图语法。

### 调用第三方库

完善支持NumPy、SciPy等第三方库。静态图模式支持np.ndarray等众多第三方库数据类型及其运算操作，支持获取调用第三方库的属性和方法，并支持通过Tensor的asnumpy()等方法与NumPy等三方库进行交互处理。也就是说，用户可以在静态图模式下调用MindSpore自身接口和算子，或者直接调用三方库的接口，也可以把它们融合在一起使用。

- 支持第三方库(如NumPy、SciPy等)的数据类型，允许调用和返回第三方库的对象。
- 支持调用第三方库的方法。
- 支持使用NumPy第三方库数据类型创建Tensor对象。
- 暂不支持对第三方库数据类型的下标索引赋值。

更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[调用第三方库](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#调用第三方库)。

### 支持自定义类的使用

用户自定义的类，没有使用`@jit_class`修饰，也没有继承`nn.Cell`。通过JIT Fallback技术方案，静态图模式下允许创建和引用自定义类的实例，可以支持直接获取和调用自定义类实例的属性和方法，并且允许修改属性(Inplace操作)。

更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[支持自定义类的使用](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#支持自定义类的使用)。

### 基础运算符支持更多数据类型

在静态图语法重载了以下运算符: ['+', '-', '*', '/', '//', '%', '**', '<<', '>>', '&', '|', '^', 'not', '==', '!=', '<', '>', '<=', '>=', 'in', 'not in', 'y=x[0]']。图模式重载的运算符详见[运算符](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph_syntax/operators.html)。列表中的运算符在输入图模式中不支持的输入类型时将使用扩展静态图语法支持，并使输出结果与动态图模式下的输出结果一致。

更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[基础运算符支持更多数据类型](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#基础运算符支持更多数据类型)。

### 基础类型

扩展对Python原生数据类型`List`、`Dictionary`、`None`的支持。更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[基础类型](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#基础类型)。

#### 支持列表就地修改操作

- 支持从全局变量中获取原`List`对象。
- 不支持对输入`List`对象进行inplace操作。
- 支持部分`List`内置函数的就地修改操作。

#### 支持Dictionary的高阶用法

- 支持顶图返回Dictionary。
- 支持Dictionary索引取值和赋值。

#### 支持使用None

`None`是Python中的一个特殊值，表示空，可以赋值给任何变量。对于没有返回值语句的函数认为返回`None`。同时也支持`None`作为顶图或者子图的入参或者返回值。支持`None`作为切片的下标，作为`List`、`Tuple`、`Dictionary`的输入。

### 内置函数支持更多数据类型

扩展内置函数的支持范围。Python内置函数完善支持更多输入类型，例如第三方库数据类型。更多内置函数的支持情况可见[Python内置函数](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph_syntax/python_builtin_functions.html)章节。

### 支持控制流

为了提高Python标准语法支持度，实现动静统一，扩展支持更多数据类型在控制流语句的使用。控制流语句是指`if`、`for`、`while`等流程控制语句。理论上，通过扩展支持的语法，在控制流场景中也支持。更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[支持控制流](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#支持控制流)。

### 支持属性设置与修改

支持更多类型的Inplace操作。之前版本只支持通过Inplace算子对Parameter类型进行值修改，在MindSpore2.1版本静态图模式下，支持了自定义类，Cell子类，jit_class类的属性修改。除了支持更改类self属性和全局变量的属性以外，也新增支持对List类型的extend()、reverse()、insert()、pop()等Inplace操作。更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[支持属性设置与修改](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#支持属性设置与修改)。

- 对自定义类对象以及第三方类型的属性进行设置与修改。
- 对Cell的self对象进行修改。
- 对静态图内的Cell对象以及jit_class对象进行设置与修改。

### 支持求导

使用JIT Fallback支持的静态图语法，同样支持其在求导中使用。更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[支持求导](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#支持求导)。

### Annotation Type

对于运行时的JIT Fallback支持的语法，会产生一些无法被类型推导出的节点，这种类型称为`Any`类型。因为该类型无法在编译时推导出正确的类型，所以这种`Any`将会以一种默认最大精度`float64`进行运算，防止其精度丢失。为了能更好的优化相关性能，需要减少`Any`类型数据的产生。当用户可以明确知道当前通过扩展支持的语句会产生具体类型的时候，我们推荐使用`Annotation @jit.typing:`的方式进行指定对应Python语句类型，从而确定解释节点的类型避免`Any`类型的生成。更多使用可见[静态图语法支持](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html)中的[Annotation Type](https://www.mindspore.cn/docs/zh-CN/master/model_train/program_form/static_graph.html#annotation-type)。

### 使用须知

在使用静态图JIT Fallback扩展支持语法时，请注意以下几点：

1. 对标动态图的支持能力，即：须在动态图语法范围内，包括但不限于数据类型等。

2. 在扩展静态图语法时，支持了更多的语法，但执行性能可能会受影响，不是最佳。

3. 在扩展静态图语法时，支持了更多的语法，由于使用Python的能力，不能使用MindIR导入导出的能力。

4. 暂不支持跨Python文件重复定义同名的全局变量，且这些全局变量在网络中会被用到。

## 动态图转静态图技术

MindSpore提供了一种在代码无需修改的情况下，直接将用户的动态图代码转换成静态图的功能——PIJit。该功能同时兼顾性能和易用性，去除动静模式切换的代价，真正做到动静统一。它基于Python字节码的分析，对Python的执行流进行图捕获，可以以静态图方式运行的子图便以静态图方式运行，Python语法不支持的子图便以动态图方式运行，同时通过修改调整字节码的方式链接静态图，达到动静混合执行。在满足易用性的前提下，尽可能地提高性能。

### PIJit包含以下功能

- 1. 图捕获：对字节码预处理，动态跟踪解释执行，识别MindSpore可入图操作，提供裂图功能保证函数（字节码）功能的正确性。
- 2. 字节码支持：当前支持Python3.7、Python3.8、Python3.9和Python3.10的字节码。
- 3. 图优化：对图中生成的字节码进行优化，包括分支裁剪、字节码筛选、函数字节码内联、常量折叠等功能。
- 4. 异常捕获机制：支持with、try-except语法。
- 5. 支持循环处理：通过模拟字节码的操作栈实现图捕获、裂图等特性。
- 6. UD分析：通过变量的user-def链分析的方法，解决部分参数类型不能作为静态图的返回值问题（函数、Bool、None），同时减少无用的入参，提高图的执行效率，减少数据的拷贝。
- 7. 副作用分析处理：弥补静态图的副作用处理上的劣势，根据不同场景，收集记录产生副作用的变量及字节码，在保证程序语义的基础上，在静态图外补充副作用的处理。
- 8. 守护门禁：门禁（Guard）记录了子图/优化进入的输入需要满足的条件，检查输入是否适合对应的子图优化。
- 9. Cache：图管理（Cache）则缓存了子图/优化和门禁（Guard）对应关系。
- 10. Dynamic Shape和Symbolic Shape: 使用input_signature支持Tensor/Tensor List/Tensor Tuple的DynamicShape和SymblicShape作为输入提示。同时支持自动多次运行后，识别Dynamic Shape。
- 11. 即时跟踪编译: 在跟踪和字节码分析过程中，支持算子等类型推导。
- 12. 自动混合精度: 支持原生mindspore.nn.Cell的自动混合精度能力。

### 使用方式

def jit(fn=None, input_signature=None, hash_args=None, jit_config=None, mode="PIJit"):

原Jit功能使用mode="PSJit"，新特性PIJit使用mode="PIJit"，jit_config传递一个参数字典，可以提供一些优化和调试选项。如：print_after_all可以打印入图的字节码和裂图信息，loop_unrolling可以提供循环展开功能，enable_dynamic_shape使能动态Shape。

### 使用限制

- 不支持在静态图模式下，运行带装饰@jit(mode=\"PIJit\")的函数，此时该装饰@jit(mode=\"PIJit\")视为无效。
- 不支持在@jit(mode=\"PSJit\")装饰的函数内部调用带装饰@jit(mode=\"PIJit\")的函数，该装饰 @jit(mode=\"PIJit\")视为无效。

